In [34]:
from gettext import npgettext
from importlib.metadata import requires
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [28]:
#import url
url = 'https://kworb.net/spotify/country/hk_daily.html'

# using requests package to download from web
req = requests.get(url)
req.encoding = 'utf-8' # super important!!!
assert req
# <Response [200]> meaning it is successful

print(req.text)
soup = BeautifulSoup (req.text, features ='html.parser')
print(soup.prettify())   # 擺得靚啲

print(soup.title.text)  #Spotify Daily Chart - Hong Kong

#obtain info. from tag <body>
table1 = soup.find('table',id='spotifydaily')
assert table1

<!DOCTYPE html>
<html lang="en"><head><title>Spotify Daily Chart - Hong Kong</title>
<link rel="stylesheet" type="text/css" href="/css/standard0002.css" />
<link rel="stylesheet" type="text/css" href="/css/tables0009.css" />
<link rel="stylesheet" type="text/css" href="/css/menus0001.css" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<meta name="description" content="A website that collects and analyzes music data from around the world. All of the charts, sales and streams, constantly updated.">
<meta name="keywords" content="music, charts, iTunes, Spotify, YouTube, artists, radio, popularity, sales, streaming, views, certifications">
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
<script src="/sorttable.js"></script>
<script>
  (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
  (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
  m=s.getElementsByTagName(o)[0];a.

In [29]:
# Obtain every title of columns with tag <th>
# <table><tr><th>ABC</th><th>DEF</th></table>
# Transformation / Mapping
# ["ABC", "DEF"]
# ["ABC1234", "DE3415"] -> ["ABC", "DE"] (map)
# [1, 5, 1, 2, 5, 6, 1] -> 21 (reduce)
# reduce(lambda acc, x: acc+x, array, 0)

headers = list( 
  map(
    lambda x: x.text, 
    table1.find_all('th')
  ) 
)

# headers=[]
# for th_cell in table1.find_all('th'):
#   title = th_cell.text
#   headers.append(title)

assert headers

In [30]:
mydata = pd.DataFrame (columns = headers)

mydata


,Pos,P+,Artist and Title,Days,Pk,(x?),Streams,Streams+,7Day,7Day+,Total


In [31]:
# mydata
# Iteration 1: mydata = [], length = 0, mydata.loc[0] = ....
# Iteration 2: mydata = [....], length = 1, mydata.loc[1] = yyyy
# Iteration 3: mydata = [....,yyyy], length = 2, mydata.loc[2] = xxxx

#create a for loop to full mydata
for row in table1.find_all('tr')[1:]:  # j 做variable, 搵 <tr> 由第2行去到最尾
    cells = row.find_all('td') # 每一個j 搵入面<td>所有野，做row_data
    cell_text = [ i.text for i in cells ]
    pos = len(mydata)
    mydata.loc[pos] = cell_text


# for line in mydata:
#     print (line.encode('utf-8'))

print(mydata)

     Pos   P+                                  Artist and Title  Days   Pk  \
0      1    =                                      MC 張天賦 - 記憶棉    94    1   
1      2    =                                     MC 張天賦 - 反對無效   234    2   
2      3    =                                     MC 張天賦 - 時候不早    41    2   
3      4    =                            COLLAR - Call My Name!    11    1   
4      5    =                    William Wei - 如果可以 - 電影"月老"主題曲    48    4   
..   ...  ...                                               ...   ...  ...   
195  196   RE                        Ronald Cheng - My Only One   408   25   
196  197   RE                         YENA - SMILEY(Feat. BIBI)     4  179   
197  198  -21                         Charmaine Fong - 你是你本身的傳奇  1892   10   
198  199   RE                                    Justin Lo - 命硬  2290   30   
199  200    =  MC 張天賦 - Can’t Stop Myself Tonight - Studio Live    34  111   

      (x?) Streams Streams+     7Day    7Day+      Total  
0   

In [32]:
import datetime 
from datetime import timedelta
yyyymmdd = datetime.datetime.today()
yyyymmdd = yyyymmdd-timedelta(days=2)
yyyymmdd = yyyymmdd.strftime('%Y%m%d')

In [33]:
mydata.to_csv(f"spotify_data_{yyyymmdd}.csv",index=False)